In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../../raw_data/State_Population_Data_2000_2019.xls', sheet_name="Sheet1")

In [3]:
data.head()

,observation_date,state,population
0,1950-01-01,ak,135.0
1,1950-01-01,al,3058.0
2,1950-01-01,ar,1908.0
3,1950-01-01,az,756.0
4,1950-01-01,ca,10677.0


In [4]:
data['day'] = data['observation_date'].apply(lambda x: x.strftime("%Y-%m-%d"))
data['month'] = data['observation_date'].apply(lambda x: x.strftime("%Y-%m"))
data['month_number'] = data['observation_date'].apply(lambda x: int(x.month))
data['year_number'] = data['observation_date'].apply(lambda x: int(x.year))

In [5]:
seasons = ['winter','winter','spring','spring','spring','summer','summer','summer','autumn','autumn','autumn','winter']
month_to_season = dict(zip(range(1,13), seasons))
data['season'] = data.month_number.map(month_to_season)
data['year_season'] = data['year_number'].astype(str)+'-'+data['season']

In [6]:
data

,observation_date,state,population,day,month,month_number,year_number,season,year_season
0,1950-01-01,ak,135.000,1950-01-01,1950-01,1,1950,winter,1950-winter
1,1950-01-01,al,3058.000,1950-01-01,1950-01,1,1950,winter,1950-winter
2,1950-01-01,ar,1908.000,1950-01-01,1950-01,1,1950,winter,1950-winter
3,1950-01-01,az,756.000,1950-01-01,1950-01,1,1950,winter,1950-winter
4,1950-01-01,ca,10677.000,1950-01-01,1950-01,1,1950,winter,1950-winter
...,...,...,...,...,...,...,...,...,...
3451,2020-01-01,vt,623.347,2020-01-01,2020-01,1,2020,winter,2020-winter
3452,2020-01-01,wa,7693.612,2020-01-01,2020-01,1,2020,winter,2020-winter
3453,2020-01-01,wi,5832.655,2020-01-01,2020-01,1,2020,winter,2020-winter
3454,2020-01-01,wv,1784.787,2020-01-01,2020-01,1,2020,winter,2020-winter


In [7]:
def value_count_func (series_):
    return len(series_.value_counts())

def state_year_df (data_, state_, year_):
    
    data_state = data_[(data_['state'] == state_) & (data_['year_number'] == year_)]
    
    data_state_season = data_state.groupby(by=['year_number']).agg({'population':'first'})
       
    data_state_season['state'] = state_
    
    return data_state_season.reset_index().set_index('state')


In [8]:
STATES_ = list(data.state.unique()) 
YEARS_ = list(data.year_number.unique())
x_ = pd.DataFrame()

for st_ in STATES_:
    for yr_ in YEARS_:
        y_ = state_year_df (data, st_, yr_)
        x_ = pd.concat([x_,y_])
    
x_ = x_.reset_index()

In [9]:
x_

,state,year_number,population
0,ak,1950,135.000
1,ak,1951,158.000
2,ak,1952,189.000
3,ak,1953,205.000
4,ak,1954,215.000
...,...,...,...
3402,wy,2016,585.243
3403,wy,2017,579.994
3404,wy,2018,579.054
3405,wy,2019,580.116


In [10]:
x_ = x_[x_['year_number']>1974].reset_index(drop=True)
y_ = x_.iloc[np.arange(len(x_)).repeat(12)].reset_index(drop=True)
months_ = pd.DataFrame({'month_number':np.arange(12)})
months_ = months_.iloc[np.tile(np.arange(len(months_)), len(x_))]. reset_index(drop = True)
merged = y_.join(months_, lsuffix='_caller', rsuffix='_other')
merged['year_month'] = merged['year_number'].astype(str)+'-'+merged['month_number'].astype(str)

In [11]:
merged

,state,year_number,population,month_number,year_month
0,ak,1975,376.170,0,1975-0
1,ak,1975,376.170,1,1975-1
2,ak,1975,376.170,2,1975-2
3,ak,1975,376.170,3,1975-3
4,ak,1975,376.170,4,1975-4
...,...,...,...,...,...
26491,wy,2020,582.328,7,2020-7
26492,wy,2020,582.328,8,2020-8
26493,wy,2020,582.328,9,2020-9
26494,wy,2020,582.328,10,2020-10


In [12]:
merged.to_csv('../../raw_data/population_by_state_year_month.csv',header=True,index=False)